# Hello Feature Class example: using the feature classes to calculate features

This example shows how to use the Radiomics package to directly instantiate the feature classes for feature extraction. 
Note that this is not the intended standard use. For an example on the standard use with feature extractor, see the `helloRadiomics` example.

In [1]:
from __future__ import print_function
import os
import collections
import SimpleITK as sitk
import numpy
import six
import radiomics
from radiomics import firstorder, glcm, imageoperations, shape, glrlm

## Getting the test case

Testing data is contained in the pyradiomics/data folder, while this file is in the pyradiomics/bin/Notebooks folder. 

The next line of code gets the location of the current path and gets the location of the data as a relative path by going up two folders ("..") and then move into the data folders ("data").

For this to work, the current active directory should be pyradiomics/bin/notebooks, which is the case if this file is run from the pyradiomics/bin/Notebooks folder.

In [2]:
testCase = 'brain1'
dataDir = os.path.join(os.path.abspath(""), "..", "..", "data")
imageName = os.path.join(dataDir, testCase + '_image.nrrd')
maskName = os.path.join(dataDir, testCase + '_label.nrrd')

if not os.path.exists(imageName):
  print('Error: problem finding input image', imageName)
if not os.path.exists(maskName):
  print('Error: problem finding input labelmap', maskName)

In [3]:
image = sitk.ReadImage(imageName)
mask = sitk.ReadImage(maskName)

## Preprocess the image

#### Extraction Settings

In [4]:
kwargs = {}
kwargs['binWidth'] = 25
kwargs['resampledPixelSpacing'] = None
# kwargs['resampledPixelSpacing'] = [3, 3, 3]  # This is an example for defining resampling (voxels with size 3x3x3mm)
kwargs['interpolator'] = 'sitkBSpline'
kwargs['verbose'] = True

#### If enabled, resample the image

In [5]:
# Resample if necessary
if kwargs['interpolator'] != None and kwargs['resampledPixelSpacing'] != None:
  image, mask = imageoperations.resampleImage(image, mask, kwargs['resampledPixelSpacing'], kwargs['interpolator'])

## Calculate features using original image

In [6]:
# Crop the image
# bb is the bounding box, upon which the image and mask are cropped
croppedImage, croppedMask, bb = imageoperations.cropToTumorMask(image, mask)

### Calculate Firstorder features

In [7]:
firstOrderFeatures = firstorder.RadiomicsFirstOrder(croppedImage, croppedMask, **kwargs)

# Set the features to be calculated
firstOrderFeatures.enableFeatureByName('Mean', True)
# firstOrderFeatures.enableAllFeatures()

In [8]:
# Print out the docstrings of the enabled features
print('Will calculate the following first order features: ')
for f in firstOrderFeatures.enabledFeatures.keys():
  print(f)
  print(eval('firstOrderFeatures.get' + f + 'FeatureValue.__doc__'))

Will calculate the following first order features: 
Mean

    Calculate the Mean Value for the image array.

    :math:`mean = \frac{1}{N}\displaystyle\sum^{N}_{i=1}{\textbf{X}(i)}`
    


In [9]:
# Calculate the features and print(out result)
print('Calculating first order features...',)
firstOrderFeatures.calculateFeatures()
print('done')

print('Calculated first order features: ')
for (key, val) in six.iteritems(firstOrderFeatures.featureValues):
  print('  ', key, ':', val)

Calculating first order features...
done
Calculated first order features: 
   Mean : 825.235436307


### Calculate Shape Features

In [10]:
shapeFeatures = shape.RadiomicsShape(croppedImage, croppedMask, **kwargs)

# Set the features to be calculated
# shapeFeatures.enableFeatureByName('Volume', True)
shapeFeatures.enableAllFeatures()

In [11]:
# Print out the docstrings of the enabled features
print('Will calculate the following shape features: ')
for f in shapeFeatures.enabledFeatures.keys():
  print(f)
  print(eval('shapeFeatures.get' + f + 'FeatureValue.__doc__'))

Will calculate the following shape features: 
Maximum3DDiameter

    Calculate the largest pairwise euclidean distance between tumor surface voxels.
    Also known as Feret Diameter.
    
Compactness2

    Calculate the Compactness (2) of the tumor region.

    :math:`compactness\ 2 = 36\pi\frac{V^2}{A^3}`

    Compactness 2 is a measure of how compact the shape of the tumor is
    relative to a sphere (most compact). It is a dimensionless measure,
    independent of scale and orientation. This is a measure of the compactness
    of the shape of the image ROI.
    
Compactness1

    Calculate the compactness (1) of the tumor region.

    :math:`compactness\ 1 = \frac{V}{\sqrt{\pi}A^{\frac{2}{3}}}`

    Compactness 1 is a measure of how compact the shape of the tumor is
    relative to a sphere (most compact). It is a dimensionless measure,
    independent of scale and orientation. Compactness 1 is defined as the
    ratio of volume to the :math:`\sqrt{\text{surface area}^3}`. This is a

In [12]:
# Calculate the features and print(out result)
print('Calculating shape features...',)
shapeFeatures.calculateFeatures()
print('done')

print('Calculated shape features: ')
for (key, val) in six.iteritems(shapeFeatures.featureValues):
  print('  ', key, ':', val)

Calculating shape features...
done
Calculated shape features: 
   Maximum3DDiameter : 65.5366145873
   Compactness2 : 0.114127701901
   Maximum2DDiameterSlice : 47.2187913633
   Sphericity : 0.485061744222
   Compactness1 : 26.7546787215
   Elongation : 1.7789885567
   SurfaceVolumeRatio : 0.392308261863
   Volume : 16412.6586914
   Flatness : 1.21918505897
   SphericalDisproportion : 2.06159321347
   Roundness : 0.61469066615
   SurfaceArea : 6438.82160378
   Maximum2DDiameterColumn : 44.5487904052
   Maximum2DDiameterRow : 61.5801767135


### Calculate GLCM Features

In [13]:
glcmFeatures = glcm.RadiomicsGLCM(croppedImage, croppedMask, **kwargs)

# Set the features to be calculated
# glcmFeatures.enableFeatureByName('SumEntropy', True)
glcmFeatures.enableAllFeatures()

In [14]:
# Print out the docstrings of the enabled features
print('Will calculate the following GLCM features: ')
for f in glcmFeatures.enabledFeatures.keys():
  print(f)
  print(eval('glcmFeatures.get' + f + 'FeatureValue.__doc__'))

Will calculate the following GLCM features: 
SumVariance

    Using coefficients :math:`p_{x+y}` and SumEntropy (SE) calculate and return the mean Sum Variance.

    :math:`sum\ variance = \displaystyle\sum^{2N_g}_{k=2}{(k-SE)^2p_{x+y}(k)}`

    Sum Variance is a measure of heterogeneity that places higher weights on
    neighboring intensity level pairs that deviate more from the mean.
    
Homogeneity1

    Calculate and return the mean Homogeneity 1.

    :math:`homogeneity\ 1 = \displaystyle\sum^{N_g}_{i=1}\displaystyle\sum^{N_g}_{j=1}{\frac{p(i,j)}{1+|i-j|}}`

    Homogeneity 1 is a measure of the similarity in intensity values for
    neighboring voxels. It is a measure of local homogeneity that increases
    with less contrast in the window.
    
Homogeneity2

    Calculate and return the mean Homogeneity 2.

    :math:`homogeneity\ 2 = \displaystyle\sum^{N_g}_{i=1}\displaystyle\sum^{N_g}_{j=1}{\frac{p(i,j)}{1+|i-j|^2}}`

    Homogeneity 2 is a measure of the similarity in inten

In [15]:
# Calculate the features and print(out result)
print('Calculating GLCM features...',)
glcmFeatures.calculateFeatures()
print('done')

print('Calculated GLCM features: ')
for (key, val) in six.iteritems(glcmFeatures.featureValues):
  print('  ', key, ':', val)

Calculating GLCM features...
done
Calculated GLCM features: 
   SumVariance : 895.891808819
   Homogeneity1 : 0.276140402104
   Homogeneity2 : 0.189156155892
   ClusterShade : -52.9707943386
   MaximumProbability : 0.00792784235012
   Idmn : 0.957796447609
   SumVariance2 : 103.142793792
   Contrast : 52.2310659277
   DifferenceEntropy : 3.79686113536
   InverseVariance : 0.188666637795
   Entropy : 8.79428086119
   Dissimilarity : 5.58932678922
   DifferenceVariance : 17.6107741076
   Idn : 0.866370546902
   Idm : 0.189156155892
   Correlation : 0.335214788202
   Autocorrelation : 292.684050471
   SumEntropy : 5.31547876648
   AverageIntensity : 17.1242601309
   Energy : 0.00290880217681
   SumSquares : 39.9781084143
   ClusterProminence : 26251.1709801
   SumAverage : 33.4497492152
   Imc2 : 0.692033706271
   Imc1 : -0.091940840043
   DifferenceAverage : 5.58932678922
   Id : 0.276140402104
   ClusterTendency : 103.142793792


### Calculate GLRLM Features

In [16]:
glrlmFeatures = glrlm.RadiomicsGLRLM(croppedImage, croppedMask, **kwargs)

# Set the features to be calculated
# glrlmFeatures.enableFeatureByName('ShortRunEmphasis', True)
glrlmFeatures.enableAllFeatures()

In [17]:
# Print out the docstrings of the enabled features
print('Will calculate the following GLRLM features: ')
for f in glrlmFeatures.enabledFeatures.keys():
  print(f)
  print(eval('glrlmFeatures.get' + f + 'FeatureValue.__doc__'))

Will calculate the following GLRLM features: 
ShortRunLowGrayLevelEmphasis

    Calculate and return the mean Short Run Low Gray Level Emphasis (SRLGLE) value for all GLRLMs.

    :math:`SRLGLE = \frac{\sum^{N_g}_{i=1}\sum^{N_r}_{j=1}{\frac{\textbf{P}(i,j|\theta)}{i^2j^2}}}{\sum^{N_g}_{i=1}\sum^{N_r}_{j=1}{\textbf{P}(i,j|\theta)}}`

    Measures the joint distribution of shorter run lengths with lower gray-level values.
    
GrayLevelVariance

    Calculate and return the Gray Level Variance (GLV) value.

    :math:`GLV = \displaystyle\sum^{N_g}_{i=1}\displaystyle\sum^{N_r}_{j=1}{p(i,j|\theta)(i - \mu)^2}`, where

    :math:`\mu = \displaystyle\sum^{N_g}_{i=1}\displaystyle\sum^{N_r}_{j=1}{p(i,j|\theta)i}`

    Measures the variance in gray level intensity for the runs.
    
LowGrayLevelRunEmphasis

    Calculate and return the mean Low Gray Level Run Emphasis (LGLRE) value for all GLRLMs.

    :math:`LGLRE = \frac{\sum^{N_g}_{i=1}\sum^{N_r}_{j=1}{\frac{\textbf{P}(i,j|\theta)}{i^2}}}{\s

In [18]:
# Calculate the features and print(out result)
print('Calculating GLRLM features...',)
glrlmFeatures.calculateFeatures()
print('done')

print('Calculated GLRLM features: ')
for (key, val) in six.iteritems(glrlmFeatures.featureValues):
  print('  ', key, ':', val)

Calculating GLRLM features...
done
Calculated GLRLM features: 
   ShortRunLowGrayLevelEmphasis : 0.00822976624416
   GrayLevelVariance : 39.118151022
   LowGrayLevelRunEmphasis : 0.00860039789166
   GrayLevelNonUniformityNormalized : 0.0451412381498
   RunVariance : 0.0847945778959
   GrayLevelNonUniformity : 175.635192315
   LongRunEmphasis : 1.22684403826
   ShortRunHighGrayLevelEmphasis : 268.974179841
   RunLengthNonUniformity : 3500.04323157
   ShortRunEmphasis : 0.955939173141
   LongRunHighGrayLevelEmphasis : 341.286579098
   RunPercentage : 0.940406463249
   LongRunLowGrayLevelEmphasis : 0.0106011704787
   RunEntropy : 4.91503800316
   HighGrayLevelRunEmphasis : 281.066493909
   RunLengthNonUniformityNormalized : 0.895049465948


## Calculate Features using Laplacian of Gaussian Filter

Calculating features on filtered images is very similar to calculating features on the original image. All filters in PyRadiomics have the same input and output signature, and there is even one for applying no filter. This enables to loop over a list of requested filters and apply them in the same piece of code. It is applied like this in the execute function in feature extractor. The input for the filters is the image, with additional keywords. If no additional keywords are supplied, the filter uses default values where applicable. It returns a [generator object](https://docs.python.org/2/reference/simple_stmts.html?#yield), allowing to define the generators to be applied before the filters functions are actually called.

### Calculate Firstorder on LoG filtered images

In [19]:
logFeatures = {}
sigmaValues = [1.0, 3.0, 5.0]
for logImage, inputImageName, inputKwargs in imageoperations.getLoGImage(image, sigma=sigmaValues, verbose=True):
  logImage, croppedMask, bb = imageoperations.cropToTumorMask(logImage, mask)
  logFirstorderFeatures = firstorder.RadiomicsFirstOrder(logImage, croppedMask, **inputKwargs)
  logFirstorderFeatures.enableAllFeatures()
  logFirstorderFeatures.calculateFeatures()
  logFeatures[inputImageName] = logFirstorderFeatures.featureValues

	Computing LoG with sigma 1
	Computing LoG with sigma 3
	Computing LoG with sigma 5


In [20]:
# Show result
for sigma, features in six.iteritems(logFeatures):
  for (key, val) in six.iteritems(features):
    laplacianFeatureName = '%s_%s' % (str(sigma), key)
    print('  ', laplacianFeatureName, ':', val)

   log-sigma-3-0-mm-3D_InterquartileRange : 103.158138275
   log-sigma-3-0-mm-3D_Skewness : -0.498386343995
   log-sigma-3-0-mm-3D_Uniformity : 0.0906478492348
   log-sigma-3-0-mm-3D_MeanAbsoluteDeviation : 64.3312024633
   log-sigma-3-0-mm-3D_Energy : 15235011555.6
   log-sigma-3-0-mm-3D_RobustMeanAbsoluteDeviation : 43.3779243984
   log-sigma-3-0-mm-3D_Median : -73.3129653931
   log-sigma-3-0-mm-3D_TotalEnergy : 60441635199.8
   log-sigma-3-0-mm-3D_Maximum : 114.296691895
   log-sigma-3-0-mm-3D_RootMeanSquared : 1919.01616706
   log-sigma-3-0-mm-3D_90Percentile : 13.9173410416
   log-sigma-3-0-mm-3D_Minimum : -354.335235596
   log-sigma-3-0-mm-3D_Entropy : 3.72121444058
   log-sigma-3-0-mm-3D_StandardDeviation : 81.9760118492
   log-sigma-3-0-mm-3D_Range : 468.63192749
   log-sigma-3-0-mm-3D_Variance : 6720.06651871
   log-sigma-3-0-mm-3D_10Percentile : -197.017340088
   log-sigma-3-0-mm-3D_Kurtosis : 3.18336583197
   log-sigma-3-0-mm-3D_Mean : -82.7355469484
   log-sigma-1-0-mm-3D_I

## Calculate Features using Wavelet filter

### Calculate Firstorder on filtered images

In [21]:
waveletFeatures = {}
for decompositionImage, decompositionName, inputKwargs in imageoperations.getWaveletImage(image):
  decompositionImage, croppedMask, bb = imageoperations.cropToTumorMask(decompositionImage, mask)
  waveletFirstOrderFeaturs = firstorder.RadiomicsFirstOrder(decompositionImage, croppedMask, **kwargs)
  waveletFirstOrderFeaturs.enableAllFeatures()
  waveletFirstOrderFeaturs.calculateFeatures()

  print('Calculated firstorder features with ', decompositionName)
  waveletFeatures[decompositionName] = waveletFirstOrderFeaturs.featureValues

Calculated firstorder features with  wavelet-LHL
Calculated firstorder features with  wavelet-LHH
Calculated firstorder features with  wavelet-HLL
Calculated firstorder features with  wavelet-LLH
Calculated firstorder features with  wavelet-HLH
Calculated firstorder features with  wavelet-HHH
Calculated firstorder features with  wavelet-HHL
Calculated firstorder features with  wavelet-LLL


In [22]:
# Show result
for decompositionName, features in six.iteritems(waveletFeatures):
  for (key, val) in six.iteritems(features):
    waveletFeatureName = '%s_%s' % (str(decompositionName), key)
    print('  ', waveletFeatureName, ':', val)

   wavelet-LLL_InterquartileRange : 550.594267427
   wavelet-LLL_Skewness : 0.228846426465
   wavelet-LLL_Uniformity : 0.0199077767278
   wavelet-LLL_MeanAbsoluteDeviation : 293.143995944
   wavelet-LLL_Energy : 75413385469.3
   wavelet-LLL_RobustMeanAbsoluteDeviation : 220.739697172
   wavelet-LLL_Median : 2244.88673609
   wavelet-LLL_TotalEnergy : 299186404755.0
   wavelet-LLL_Maximum : 3180.63918677
   wavelet-LLL_RootMeanSquared : 4269.54365408
   wavelet-LLL_90Percentile : 2739.69052111
   wavelet-LLL_Minimum : 1468.07724103
   wavelet-LLL_Entropy : 5.78300489052
   wavelet-LLL_StandardDeviation : 350.172190209
   wavelet-LLL_Range : 1712.56194574
   wavelet-LLL_Variance : 122620.562796
   wavelet-LLL_10Percentile : 1812.68473489
   wavelet-LLL_Kurtosis : 2.27365643067
   wavelet-LLL_Mean : 2255.1595095
   wavelet-HHH_InterquartileRange : 20.3192746422
   wavelet-HHH_Skewness : -0.0688112737241
   wavelet-HHH_Uniformity : 0.382919979814
   wavelet-HHH_MeanAbsoluteDeviation : 13.07